# **Playing Fetch with Spot**

The goal of this script is to add pretrained models to spot which can detect objects.
This should allow a user to play fetch with spot.

## Collect Images

Our goal is to collect images of the dog in as many environments as possible.

In [1]:
# Activate environment (PowerShell): spot_env\Scripts\activate

# Install OpenCV's python bindings in your virtualenv:
!py.exe -3 -m pip install opencv-python==4.5.* -q

# Install numpy
!pip install numpy -q

# cd ./Object detection/fetch

# Running our code to capture images
!py.exe -3 capture_images.py 192.168.80.3 --image-source right_fisheye_image --folder dogtoy/images

## Label Images

To label our images, we’ll use an open source off-the-shelf labeling program. 

In [4]:
!py.exe -3 -m pip install labelme -q
!labelme

In the labelImg display, open the image directory with File > Open Dir

The main keys we'll use are:
- d     Next image
- a     Previous image
- w     Create a box
- del	Delete the selected box

Use d to get to an image that has the dog toy in it. Press w then click-and-drag to make a box on the toy. Enter dogtoy into the pop-up and press OK.

Tip: If the box refuses to be a rectangle, press Ctrl to disable that mode.

## Training the model

#### Installations

Install NVIDIA drivers, CUDA, and cuDNN

- <font color='Yellow'>What is NVIDIA?</font>

NVIDIA is known for developing integrated circuits, which are used in everything from electronic game consoles to personal computers (PCs). The company is a leading manufacturer of high-end graphics processing units (GPUs).

A driver is a software that enables the operating system to communicate with hardware or a device. Thus, for an NVIDIA graphics driver, it allows the OS to communicate with the video card, ensuring that it functions properly.

You need to have a real, physical NVIDIA graphics card in order to install NVIDIA drivers.

Link: https://helpx.adobe.com/x-productkb/multi/drivers-video-win-nvidia.html

- <font color='Yellow'>What is CUDA?</font>

NVIDIA provides something called the Compute Unified Device Architecture (CUDA), which is crucial for supporting the various deep learning applications. 

CUDA is a framework that allows people with a Nvidia Graphics Card to use GPU acceleration when it comes to deep learning. 

Link: https://developer.nvidia.com/cuda-downloads

- <font color='Yellow'>What is cuDNN?</font>

The cuDNN library is a library optimized for CUDA containing GPU implementations. Think of cuDNN as a library for Deep Learning using CUDA and CUDA as a way to talk to the GPU.

Link: https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html


Install TensorFlow

Activate virtual environment (copy/paste to terminal):

spot_env\Scripts\activate

In [9]:
# Tensorflow likes to install a non-GUI version of OpenCV, which will cause us problems later
# We can safely uninstall it because we already installed OpenCV.

!py.exe -3 -m pip install --upgrade pip
!py.exe -3 -m pip install tensorflow==2.3.1 tensorboard==2.3.0 tf-models-official==2.3.0 pycocotools lvis
!py.exe -3 -m pip uninstall opencv-python-headless

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.3.1


In [ ]:
# Test TensorFlow and CUDA installation (you should have at least 1 GPU as a result):

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Install TensorFlow Object Detection API

In [ ]:
# Download our package with the precompiled files and save it in the fetch folder
unzip models-with-protos.zip

# Install the object detection API:
# cd models-with-protos/research
!py.exe -3 -m pip install .

## Prepare Training Data

First convert JSON files to XML files

In [ ]:
!py.exe -3 -m pip install json2xml -q

In [11]:
from json2xml import json2xml
from json2xml.utils import readfromurl, readfromstring, readfromjson

# get the xml from an URL that return json
data = readfromurl("https://coderwall.com/vinitcool76.json")
print(json2xml.Json2xml(data).to_xml())

# get the xml from a json string
data = readfromstring(
    '{"login":"mojombo","id":1,"avatar_url":"https://avatars0.githubusercontent.com/u/1?v=4"}'
)
print(json2xml.Json2xml(data).to_xml())

# get the data from an URL
data = readfromjson("examples/licht.json")
print(json2xml.Json2xml(data).to_xml())

ModuleNotFoundError: No module named 'json2xml'


Partition the data into training and test sets. Download the script split_dataset.py and put it in the ~/fetch folder. Run the script to copy the XML label files into the train and test folders, splitting them up randomly.

In [ ]:
# cd ~/fetch
!python split_dataset.py --labels-dir dogtoy/annotations/ --output-dir dogtoy/annotations/ --ratio 0.9